In [1]:
# Import packages
import math

In [2]:
# Define physical properties
length = 8  # m
delta_h = 12  # in.
thickness = 0.5  # mm
D_i = 0.8  # cm
D_o = 1.2  # cm
minor_losses = 25
V_wort = 6  # gal
T_0_wort = 200  # F
T_f_wort = 70  # F
T_city = 50  # F
P_water = 80  # psi

In [3]:
# Define material properties
k_tube = 15  # W/m*K
Nu = 50  # both sides
P_atm = 101325  # Pa
g = 9.81  # m/s^2

In [4]:
# Unit conversion functions
def in_to_m(measurement):
    return 0.0254 * measurement

def mm_to_m(measurement):
    return measurement / 1000

def cm_to_m(measurement):
    return measurement / 100

def gal_to_m3(measurement):
    return measurement * 0.00378541

def f_to_c(measurement):
    return (measurement - 32) * 5 / 9

def psi_to_pa(measurement):
    return measurement * 6894.76

In [5]:
# Temperature dependent water properties
def get_mu(temp):
    mu = 0 * temp + 8.9e-4  # Pa*s
    return mu

def get_rho(temp):
    rho = 0 * temp + 1000  # kg/m^3
    return rho

def get_cp(temp):
    cp = temp * 0 + 4200  # J/kg*K
    return cp

def get_k_fluid(temp):
    k_fluid = 0 * temp + 0.6  # W/m*K
    return k_fluid

In [6]:
# Pump curves fit to polynomial in form c1 + c2Q + c3Q^2 + c4Q^3
def pump_curve_1():
    c1 = 1.4143  # m
    c2 = -0.0813  # m/lpm
    c3 = -0.0013  # m/lpm^2
    c4 = -0.0005   # m/lpm^3
    return c1, c2, c3, c4

def pump_curve_2():
    c1 = 2.8185
    c2 = -0.1978
    c3 = -0.0143
    c4 = -0.0052
    return c1, c2, c3, c4
    
def pump_curve_3():
    c1 = 2.3248
    c2 = -0.1071
    c3 = -0.0045
    c4 = -0.0002
    return c1, c2, c3, c4

# Calculate the intersection of the h_req curve with the pump curve
def intersection(pump_num, input_curve):
    return 1

In [7]:
# Convert to SI units
delta_h = in_to_m(delta_h)
thickness = mm_to_m(thickness)
D_i = cm_to_m(D_i)
D_o = cm_to_m(D_o)
V_wort = gal_to_m3(V_wort)
T_0_wort = f_to_c(T_0_wort)
T_f_wort = f_to_c(T_f_wort)
T_city = f_to_c(T_city)
P_water = psi_to_pa(P_water)

# Derived values
D_h_out = D_o - (D_i + 2 * thickness)
rho_city = get_rho(T_city)
mu_city = get_mu(T_city)

In [12]:
# Reynolds Number Equation
def reynolds(rho, v, D, mu):
    return rho * v * D / mu

# Friction Factor Equations
def friction_factor(Re):
    return (0.79 * math.log(Re) - 1.64) ** -2

# Calculates the mean flow velocity
def u_m(pressure_grad, hydraulic_diameter, friction_factor, density):
    return math.sqrt(2 * pressure_grad * hydraulic_diameter / (friction_factor * density))

# Calculates the mass flow rate
def m_dot(velocity, area, density):
    return velocity * density * area

In [30]:
# function to calculate flow rate of the city water
def outer_flow(P_outside, f_guess):
    delta_p_city = P_outside - P_atm # Pa
    pressure_gradient = delta_p_city / length  # Pa/m
    
    # Iterate until a solution is found
    converged = False
    i = 0
    v_out = u_m(pressure_gradient, D_h_out, f_guess, rho_city)  # m/s
    print(f"Starting with f_guess = {f_guess}, v_out = {v_out}\n")
    print("Update Loop:")
    while not converged:
        Re_out = reynolds(rho_city, v_out, D_h_out, mu_city)
        f_update = friction_factor(Re_out)
        df = abs(f_update - f_guess)
        if df <= f_guess * 0.1:
            converged = True
            f_out = f_update
            Re_out = reynolds(rho_city, v_out, D_h_out, mu_city)
        else:
            f_guess = f_update
        v_out = u_m(pressure_gradient, D_h_out, f_update, rho_city)  # m/s
        i += 1
        print(f"{i}: f = {f_update}, v = {v_out}")
    
    A_out = math.pi * ((D_o ** 2) / 4 - ((D_i + 2 * thickness) ** 2) / 4)
    m_dot_out = m_dot(v_out, A_out, rho_city)
    flow_type = "Turbulent" if Re_out >= 3500 else "Laminar"
    print(f"\nConverged in {i} iterations")
    print(f"Friction Factor: {f_out:.5f}")
    print(f"Reynolds Number: {Re_out:.1f}, flow is {flow_type}")
    print(f"Mean Velocity: {v_out:.2f} m/s")
    print(f"Mass Flow: {m_dot_out:.3f} kg/s")
    
    return m_dot_out, v_out, Re_out, f_out

In [31]:
# Get city water flow properties
m_dot_out, v_out, Re_out, f_out = outer_flow(P_water, 0.04)

Starting with f_guess = 0.04, v_out = 2.9055629833132164

Update Loop:
1: f = 0.0316642697587198, v = 3.265694473327893
2: f = 0.03064913563967337, v = 3.3193356787000923

Converged in 2 iterations
Friction Factor: 0.03065
Reynolds Number: 11008.0, flow is Turbulent
Mean Velocity: 3.32 m/s
Mass Flow: 0.164 kg/s


In [11]:
# Head Loss Equation
def head_loss(f, L, D, K, v, g):
    return (f * (L / D) + K) * (v ** 2) / (2 * g)

In [ ]:
# Get convection coefficient using Nu
def convection_coefficient(nusselt, k_fluid, D):
    return nusselt * k_fluid / D

# get total thermal resistance
def thermal_resistance(d_out, d_in, t_tube, nusselt, k_tube, temperature):
    # Calculate values for the inside
    A_in = (math.pi * d_in ** 2) / 4
    k_in = get_k_fluid(temperature)
    h_in = convection_coefficient(nusselt, k_in, d_in)
    
    # Calculate values for the outside
    A_out = math.pi * ((d_out ** 2) / 4 - ((d_in + 2 * t_tube) ** 2) / 4)
    k_out = get_k_fluid(T_city)
    h_out = convection_coefficient(nusselt, k_out, d_out)
    
    # Check if I need to add length terms
    R_tot = 1 / (h_in * A_in) + math.log((d_in + 2 * t_tube)/d_in) / (2 * math.pi * k_tube) + 1 / (h_out * A_out)
    return R_tot